In [ ]:
# %conda install imutils

In [ ]:
import numpy as np
import cv2
import imutils
import sys
from imutils.video import VideoStream
import time

In [ ]:
# define names of each possible ArUco tag OpenCV supports
ARUCO_DICT = {
	"DICT_4X4_50": cv2.aruco.DICT_4X4_50,
	"DICT_4X4_100": cv2.aruco.DICT_4X4_100,
	"DICT_4X4_250": cv2.aruco.DICT_4X4_250,
	"DICT_4X4_1000": cv2.aruco.DICT_4X4_1000,
	"DICT_5X5_50": cv2.aruco.DICT_5X5_50,
	"DICT_5X5_100": cv2.aruco.DICT_5X5_100,
	"DICT_5X5_250": cv2.aruco.DICT_5X5_250,
	"DICT_5X5_1000": cv2.aruco.DICT_5X5_1000,
	"DICT_6X6_50": cv2.aruco.DICT_6X6_50,
	"DICT_6X6_100": cv2.aruco.DICT_6X6_100,
	"DICT_6X6_250": cv2.aruco.DICT_6X6_250,
	"DICT_6X6_1000": cv2.aruco.DICT_6X6_1000,
	"DICT_7X7_50": cv2.aruco.DICT_7X7_50,
	"DICT_7X7_100": cv2.aruco.DICT_7X7_100,
	"DICT_7X7_250": cv2.aruco.DICT_7X7_250,
	"DICT_7X7_1000": cv2.aruco.DICT_7X7_1000,
	"DICT_ARUCO_ORIGINAL": cv2.aruco.DICT_ARUCO_ORIGINAL,
	"DICT_APRILTAG_16h5": cv2.aruco.DICT_APRILTAG_16h5,
	"DICT_APRILTAG_25h9": cv2.aruco.DICT_APRILTAG_25h9,
	"DICT_APRILTAG_36h10": cv2.aruco.DICT_APRILTAG_36h10,
	"DICT_APRILTAG_36h11": cv2.aruco.DICT_APRILTAG_36h11
}

In [ ]:
output = "/content/drive/MyDrive/robotics_club/tags/"

def generate(id,dict_type,dim):
  # verify that the supplied ArUCo type is supported by OpenCV
  if ARUCO_DICT.get(dict_type, None) is None:
    print("[INFO] ArUCo tag of '{}' is not supported".format(dict_type))
    sys.exit(0)

  # load the ArUCo dictionary
  arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[dict_type])

  # draw the ArUCo tag on the output image
  print("[INFO] generating ArUCo tag type '{}' with ID '{}'".format(dict_type, id))
  tag = np.zeros((dim, dim, 1), dtype="uint8")
  cv2.aruco.drawMarker(arucoDict, id, dim, tag, 1)
  # write the generated ArUCo tag to disk and then display it to our screen
  cv2.imwrite(output+'id_{}.png'.format(id), tag)
  cv2.waitKey(0)

In [ ]:
generate(74,"DICT_5X5_100",300)
generate(52,"DICT_5X5_100",300)
generate(13,"DICT_5X5_100",300)
generate(35,"DICT_5X5_100",300)
generate(96,"DICT_5X5_100",300)

In [ ]:
def detect_video(dict_type):

  if ARUCO_DICT.get(dict_type, None) is None:
    print("[INFO] ArUCo tag of '{}' is not supported".format(dict_type))
    sys.exit(0)
  
  print("[INFO] detecting '{}' tags...".format(dict_type))
  arucoDict = cv2.aruco.Dictionary_get(ARUCO_DICT[dict_type])
  arucoParams = cv2.aruco.DetectorParameters_create()
    
  print("[INFO] starting video stream...")
  vs = VideoStream(src=0).start()
  time.sleep(2.0)

  # loop over the frames from the video stream
  while True:
    frame = vs.read()
    frame = imutils.resize(frame, width=1000)
    # detect ArUco markers in the input frame
    (corners, ids, rejected) = cv2.aruco.detectMarkers(frame,
                              arucoDict, parameters=arucoParams)
    
    if len(corners) > 0:
      
      ids = ids.flatten()
      
      for (markerCorner, markerID) in zip(corners, ids):
        # extract the marker corners 
        corners = markerCorner.reshape((4, 2))
        (topLeft, topRight, bottomRight, bottomLeft) = corners
        # convert each of the (x, y)-coordinate pairs to integers
        topRight = (int(topRight[0]), int(topRight[1]))
        bottomRight = (int(bottomRight[0]), int(bottomRight[1]))
        bottomLeft = (int(bottomLeft[0]), int(bottomLeft[1]))
        topLeft = (int(topLeft[0]), int(topLeft[1]))

        # draw the bounding box of the ArUCo detection
        cv2.line(frame, topLeft, topRight, (0, 255, 0), 2)
        cv2.line(frame, topRight, bottomRight, (0, 255, 0), 2)
        cv2.line(frame, bottomRight, bottomLeft, (0, 255, 0), 2)
        cv2.line(frame, bottomLeft, topLeft, (0, 255, 0), 2)
        # compute and draw the center (x, y)-coordinates of the
        # ArUco marker
        cX = int((topLeft[0] + bottomRight[0]) / 2.0)
        cY = int((topLeft[1] + bottomRight[1]) / 2.0)
        cv2.circle(frame, (cX, cY), 4, (0, 0, 255), -1)
        # draw the ArUco marker ID on the frame
        cv2.putText(frame, str(markerID),
          (topLeft[0], topLeft[1] - 15),
          cv2.FONT_HERSHEY_SIMPLEX,
          0.5, (0, 255, 0), 2)
        
    # show the output frame
    cv2.imshow("Frame", frame)
    key = cv2.waitKey(1) & 0xFF
    # if the `q` key was pressed, break from the loop
    if key == ord("q"):
      break

  # do a bit of cleanup
  cv2.destroyAllWindows()
  vs.stop()

In [ ]:
detect_video("DICT_5X5_100")